# Preprocessing for Machine Learning 
* Standardizing Data
* Feature Engineering
*

# Standarization
for models who have high variance, continuous and on different scales, model linear space, linearly assumptions
* log normalization: high variance
* Scaling: different scales, linear chracteristics, 

In [10]:
import pandas as pd
import numpy as np
data_boston=pd.read_csv('HousingData.csv')
print(data_boston.columns)
data_boston.dropna(inplace=True)  #just for simplicity

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')


## log normalization

In [11]:
print('variance before log normalization',data_boston.TAX.var())
#lets reduce de variance of just one column (TAX) whic has he highest variance
data_boston['TAX_log']=np.log(data_boston.TAX)
print('variance after log normalization',data_boston.TAX_log.var())

variance before log normalization 28329.070355588272
variance after log normalization 0.15683393142975663


## Scaling

In [12]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
data_sclaled=pd.DataFrame(scaler.fit_transform(data_boston),columns=data_boston.columns)
print('variance before scaling',data_boston.TAX.var())
print('variance after scaling', data_sclaled.TAX.var())

variance before scaling 28329.070355588272
variance after scaling 1.0025445292620867


## Modeling data

In [13]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split 
X=data_boston.drop(columns=['MEDV'])
y=data_boston['MEDV']
knn=KNeighborsRegressor()

#### non-scaled data
# Split the dataset and labels into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y)

# Fit the k-nearest neighbors model to the training data
knn.fit(X_train,y_train)

# Score the model on the test data
print('Score for non scaled data',knn.score(X_test,y_test))

#### scaled data
# Create the scaling method.
ss = StandardScaler()

# Apply the scaling method to the dataset used for modeling.
X_scaled = ss.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)

# Fit the k-nearest neighbors model to the training data.
knn.fit(X_train,y_train)

# Score the model on the test data.
print('Score for scaled data',knn.score(X_test,y_test))

Score for non scaled data 0.5071982781054267
Score for scaled data 0.5511932867695652


# Feature engineering
* Creation of new features based on existing features 
* Insight into relationships between features 
* Extract and expand data 
* Dataset-dependent


## Encoding categorical variables 

### Binary encoding

In [14]:
from sklearn.preprocessing import LabelEncoder
hiking=pd.read_json('https://assets.datacamp.com/production/repositories/1816/datasets/4f26c48451bdbf73db8a58e226cd3d6b45cf7bb5/hiking.json')

# Set up the LabelEncoder object
enc = LabelEncoder()

# Apply the encoding to the "Accessible" column
hiking['Accessible_enc']=enc.fit_transform(hiking['Accessible'])

# Compare the two columns
print(hiking[['Accessible','Accessible_enc']].head())

  Accessible  Accessible_enc
0          Y               1
1          N               0
2          N               0
3          N               0
4          N               0


### One-Hot Encoding

creates different columns for each categorical value from a feature

In [15]:
volunteer=pd.read_csv('https://assets.datacamp.com/production/repositories/1816/datasets/668b96955d8b252aa8439c7602d516634e3f015e/volunteer_opportunities.csv')
# Transform the category_desc column
category_enc = pd.get_dummies(volunteer["category_desc"])

# Take a look at the encoded columns
print(category_enc.head())

   Education  Emergency Preparedness  Environment  Health  \
0          0                       0            0       0   
1          0                       0            0       0   
2          0                       0            0       0   
3          0                       0            0       0   
4          0                       0            1       0   

   Helping Neighbors in Need  Strengthening Communities  
0                          0                          0  
1                          0                          1  
2                          0                          1  
3                          0                          1  
4                          0                          0  


## Numerical Values

In [16]:
# Create a list of the columns to average
run_columns = ["run1", "run2", "run3", "run4", "run5"]

# Use apply to create a mean column
#running_times_5k["mean"] = running_times_5k.apply(lambda row: row[run_columns].mean(), axis=1)

# Take a look at the results
#print(running_times_5k)

## FEature  engineering for text


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
nb=GaussianNB()
# Take the title text
title_text = volunteer["title"]

# Create the vectorizer method
tfidf_vec = TfidfVectorizer()

# Transform the text into tf-idf vectors

#text_tfidf = tfidf_vec.fit_transform(title_text)


# Split the dataset according to the class distribution of category_desc
y = volunteer["category_desc"]

#X_train, X_test, y_train, y_test = train_test_split(text_tfidf.toarray(), y, stratify=y)

# Fit the model to the training data
#nb.fit(X_train,y_train)

# Print out the model's accuracy
#print(nb.score(X_test,y_test))

## Selecting features using vectors

# Complete pre-processing

In [18]:
uf=pd.read_csv('https://assets.datacamp.com/production/repositories/1816/datasets/a5ebfe5d2ed194f2668867603b563963af4769e9/ufo_sightings_large.csv')
